
# Nivel 3, Semana 6: Algoritmos Variacionales y QAOA

**Audiencia:** Licenciatura / Avanzado

**Objetivo:** Introducir el modelo híbrido de computación cuántica-clásica. Formalizar la estructura de los Algoritmos Variacionales, específicamente el ansatz QAOA. Conectar esto con las Primitivas de Qiskit (Estimator) utilizadas para ejecutarlos.

## 1. Puente desde la Semana 5: ¿Por qué necesitamos un nuevo método?

En la Semana 5, aprendimos sobre el efecto devastador del **ruido** y la **decoherencia**.

Las matemáticas formales de `Statevector` de las Semanas 1-4 solo funcionan para sistemas ideales y perfectos. Los dispositivos reales "NISQ" (Quantum Intermedio Ruidoso) pierden su "cuanticidad" (coherencia) muy rápido.

**Por lo tanto,** *no podemos* ejecutar algoritmos largos y complejos (como el algoritmo de Shor) que requieren millones de compuertas en la actualidad, ya que no existe un avance que permita sostener el funcionamiento de millones de compuertas. El ruido destruiría la respuesta.

**La solución: El ciclo híbrido cuántico-clásico**. En vez de un solo algoritmo cuántico largo, usamos una combinación de *2 tipos de computadora*:

1. Una **computadora cuántica** ejecuta un circuito muy *superficial* (corto) con parámetros ajustables. Su único trabajo es preparar un "estado de prueba" y medir un solo valor (el "costo").

2. Una **computadora clásica** hace todo el "razonamiento". Toma el costo, actualiza los parámetros y le indica a la computadora cuántica que lo intente de nuevo, repitiendo el proceso en un ciclo.

Esta es la idea central de todos los **Algoritmos Variacionales**.

## 2. El Modelo de Algoritmo Variacional

Los algoritmos variacionales son una clase amplia de algoritmos híbridos. El **Solucionador Variacional de Eigenvalores Cuánticos (VQE)** es la versión más general, diseñada para encontrar la energía del estado fundamental de *cualquier* Hamiltoniano (usado comúnmente en química cuántica). QAOA, en el que nos enfocamos, es un tipo específico de VQE adaptado para problemas de optimización clásica.

Este ciclo híbrido es un proceso de optimización, que se relaciona directamente con los conceptos de Nivel 1 y 2 de "encontrar el punto más bajo en un valle".

El ciclo funciona así:

1. **Optimizador clásico:** Elige un conjunto inicial de parámetros (ángulos), $\vec{\theta}$.

2. **Computadora cuántica (QPU):**
   * Prepara un estado simple (por ejemplo, $|+\rangle^{\otimes n}$).
   * Ejecuta el circuito "Ansatz" $U(\vec{\theta})$ (nuestro circuito superficial y parametrizado).
   * Esto prepara el estado de prueba: $|\psi(\vec{\theta})\rangle = U(\vec{\theta})|+\rangle^{\otimes n}$.

3. **Qiskit Estimator (Primitiva):**
   * Mide el **valor esperado** (o "costo") de este estado respecto a un Hamiltoniano problema, $H_P$.
   * Calcula: $C = \langle\psi(\vec{\theta})| H_P |\psi(\vec{\theta})\rangle$.

4. **Optimizador clásico:**
   * Recibe el número $C$.
   * Verifica si $C$ es el mínimo. Si no, usa un algoritmo clásico (por ejemplo, COBYLA, SPSA o descenso de gradiente) para elegir un *nuevo* conjunto de parámetros $\vec{\theta}'$.
   * El ciclo se repite hasta que el costo $C$ se minimiza.

### El Optimizador Clásico en la Práctica

Aunque el descenso de gradiente es válido, calcular gradientes en una computadora cuántica (por ejemplo, usando la "regla de cambio de parámetro") puede ser muy lento y requiere muchas ejecuciones extra de circuitos.

En la práctica, los optimizadores **sin gradiente** son muy comunes en Qiskit porque son más robustos al ruido:

* **COBYLA:** Un optimizador ampliamente usado y confiable.
* **SPSA:** Un optimizador potente para hardware real y ruidoso. Estima el gradiente para todos los $2p$ parámetros usando solo dos mediciones, lo que lo hace muy eficiente.

La computadora cuántica solo proporciona el valor de costo $C$. La computadora clásica realiza toda la optimización.

## 3. Ejemplo QAOA: Max-Cut y el Hamiltoniano de Costo

Los algoritmos variacionales están diseñados para resolver problemas de optimización. El ejemplo clásico es **Max-Cut**. Primero, debemos definir un Hamiltoniano de Costo para resolver el problema Max-Cut. Antes de profundizar en el Hamiltoniano de Costo, discutimos lo siguiente:

**¿Qué es el "costo"?** (Idea de informática): En un problema de optimización como Max-Cut, el "costo" es solo una puntuación. En esta sección, diseñamos que una puntuación baja (costo) es buena (muchas aristas cortadas), y una puntuación alta es mala (muchas aristas sin cortar). Para la cadena de bits $01$, el costo es bajo. Para la cadena $00$, el costo es alto.

**¿Qué es la "energía"?** (Idea de física): En mecánica cuántica, la "energía" es una propiedad física de un estado, determinada por su Hamiltoniano $(H)$. Una regla fundamental de la física es que los sistemas siempre intentan asentarse en su estado de energía más bajo, llamado "estado fundamental".

### Hamiltoniano de Costo

El **Hamiltoniano de Costo** (o Hamiltoniano del Problema, $H_P$) es un operador cuántico que *diseñamos* para traducir un problema de optimización clásico en un problema físico.

Se construye de modo que su **estado de energía más bajo (su "estado fundamental") corresponde a la mejor solución posible** al problema clásico.

* Encontrar la **mejor solución** (por ejemplo, el Max-Cut) se convierte en...
* ...encontrar el **estado fundamental** del Hamiltoniano $H_P$.

El **"costo"** de cualquier solución de prueba (una cadena de bits) es simplemente la energía (el valor esperado) de ese estado: $C = \langle\psi| H_P |\psi\rangle$. Menor energía significa mejor solución.

### Hamiltoniano de Costo (El método para Max-Cut)

**Paso 1. Mapear el problema a qubits:**
* **Problema:** Dividir los nodos en dos conjuntos (0 o 1) para *maximizar* las aristas cortadas.
* **Mapeo:** Un nodo = Un qubit.
  * Medir $|0\rangle$ implica que el nodo está en el conjunto 0.
  * Medir $|1\rangle$ implica que el nodo está en el conjunto 1.

**Paso 2. Asignar penalizaciones/recompensas de energía:**
* Nuestro objetivo es *maximizar* las aristas cortadas. Esto es igual a *minimizar* las aristas sin cortar.
* Una **arista sin cortar** (qubits en $|00\rangle$ o $|11\rangle$) es "mala". Le damos una penalización de energía *alta*: **+1**.
* Una **arista cortada** (qubits en $|01\rangle$ o $|10\rangle$) es "buena". Le damos una recompensa de energía *baja*: **-1**.

**Paso 3. Encontrar el operador cuántico:**
* Necesitamos un operador que dé `+1` para $|00\rangle$/$|11\rangle$ y `-1` para $|01\rangle$/$|10\rangle$. El operador $Z_i Z_j$ es perfecto para esto:
  * $Z_i Z_j |00\rangle = (+1)(+1)|00\rangle = \mathbf{+1} |00\rangle$  *(Costo alto)*
  * $Z_i Z_j |11\rangle = (-1)(-1)|11\rangle = \mathbf{+1} |11\rangle$  *(Costo alto)*
  * $Z_i Z_j |01\rangle = (+1)(-1)|01\rangle = \mathbf{-1} |01\rangle$  *(Costo bajo)*
  * $Z_i Z_j |10\rangle = (-1)(+1)|10\rangle = \mathbf{-1} |10\rangle$  *(Costo bajo)*

**Paso 4. Sumar todos los costos:**
* Para este problema, no consideramos el peso de cada nodo. Entonces, el costo total para todo el grafo es la suma de los costos de cada arista individual. Esto nos da la fórmula final:

  $$
  H_P = \sum_{(i, j) \in \text{aristas}} Z_i Z_j
  $$

* Por lo tanto, **minimizar** la energía $\langle H_P \rangle$ es matemáticamente idéntico a **maximizar el número de cortes**.

La derivación y los pasos anteriores se presentan de la forma más simple posible. La derivación completa inicia definiendo una función problema $f(x)$, luego mapeando a la notación de problemas de Optimización Binaria Cuadrática Sin Restricciones **(QUBO)**, y finalmente al Hamiltoniano Ising. Para más información, consulta:  
https://quantum.cloud.ibm.com/docs/en/tutorials/quantum-approximate-optimization-algorithm

## 4. El Ansatz Formal de QAOA

El "Ansatz" es el nombre formal para el circuito superficial y parametrizado ($U(\vec{\theta})$) que ejecuta la computadora cuántica. Para QAOA, los parámetros son $\vec{\beta}$ y $\vec{\gamma}$.

Además, para un sistema cuántico con Hamiltoniano $H$ que evoluciona durante un tiempo $t$, sufre una transformación unitaria $U = e^{-iHt}$ (asumiendo $\hbar=1$).

El ansatz se construye en $p$ capas (donde $p$ es un entero pequeño, por ejemplo, $p=1, 2, 3...$):

1. **Estado inicial:** Comenzamos en una superposición igual de todas las posibles respuestas: $|+\rangle^{\otimes n}$.

2. **Capa 1 (Costo):** Aplicar el **Hamiltoniano del Problema** $H_P$ durante un tiempo $\gamma_1$.
   * $U_P(\gamma_1) = e^{-i\gamma_1 H_P} = \prod_{(i,j)} e^{-i\gamma_1 Z_i Z_j}$
